In [2]:
import cv2
import numpy as np
import math

In [3]:
path = "C:\\Users\\1\\Desktop\\picture\\coffe.jpg"  # "C:/Users/1/Desktop/picture/Venti/Venti.jpg"
img_l = cv2.imread(path)

path_GT = "C:/Users/1/Desktop/picture/Venti/Venti_GT.jpg"
img_h = cv2.imread(path_GT)

img_l.shape  # 行，列

(591, 474, 3)

In [4]:
new_data_HR_3D = cv2.resize(img_l,
                                    dsize=None,
                                    fx=2,
                                    fy=2,
                                    interpolation=cv2.INTER_LINEAR)
new_data_HR_3D.shape

(1182, 948, 3)

In [ ]:
# 最近邻插值算法
size = (2400, 2400)
new_img_NEAREST = cv2.resize(img_l, size, interpolation=cv2.INTER_NEAREST)
cv2.imwrite("C:/Users/1/Desktop/picture/Venti/Venti_H_INTER_NEAREST.jpg", new_img_NEAREST)

In [36]:
# 最近邻插值算法
size = (2400, 2400)
new_img_NEAREST = cv2.resize(img_l, size, interpolation=cv2.INTER_NEAREST)
cv2.imwrite("C:/Users/1/Desktop/picture/Venti/Venti_H_INTER_NEAREST.jpg", new_img_NEAREST)

True

In [37]:
# 双线性插值算法
new_img_LINEAR = cv2.resize(img_l, size, interpolation=cv2.INTER_LINEAR)
cv2.imwrite("C:/Users/1/Desktop/picture/Venti/Venti_H_INTER_LINEAR.jpg", new_img_LINEAR)

True

In [38]:
# 基于4px X 4px邻域的三次插值算法
new_img_CUBIC = cv2.resize(img_l, size, interpolation=cv2.INTER_CUBIC)
cv2.imwrite("C:/Users/1/Desktop/picture/Venti/Venti_H_INTER_CUBIC.jpg", new_img_CUBIC)

True

In [39]:
# compute PSNR
def psnr(img1, img2):
    mse = np.mean((img1-img2) ** 2)
    if mse < 1.0e-10:
        return 100
    return 10 * math.log10(255.0**2 / mse)

In [40]:
print('INTER_NEAREST-最近邻插值 PSNR',format(psnr(img_h, new_img_NEAREST)))
print('INTER_LINEAR-双线性插值 PSNR',format(psnr(img_h, new_img_LINEAR)))
print('INTER_CUBIC-三次插值 PSNR',format(psnr(img_h, new_img_CUBIC)))

INTER_NEAREST-最近邻插值 PSNR 30.207982912944278
INTER_LINEAR-双线性插值 PSNR 30.23830465782856
INTER_CUBIC-三次插值 PSNR 30.20907399890178


In [42]:
# Bicubic
def base_function(x, a=-0.5):
    Wx = 0
    if np.abs(x) <= 1:
        Wx = (a+2) * (np.abs(x)**3) - (a+3)*(np.abs(x))**2 + 1
    elif 1 <= np.abs(x) <= 2:
        Wx = a * (np.abs(x)**3) - 5*a*(np.abs(x))**2 + 8*a*np.abs(x) - 4*a
    else:
        print('x的取值范围不正确')
    return Wx

def padding(img):
    h, w, c = img.shape
    print('图像的尺寸为 {}'.format(img.shape))
    pad_imgge = np.zeros((h+4, w+4, c))
    pad_imgge[2:h+2, 2:w+2] = img
    return pad_imgge

def bicubic(img, sacle, a=-0.5):
    print('Doing bicubic')
    h, w, color = img.shape
    img = padding(img)
    nh = h * sacle
    nw = h * sacle
    new_img = np.zeros((nh, nw, color))

    for c in range(color):
        for i in range(nw):
            for j in range(nh):
                px = i / sacle + 2
                py = j / sacle + 2
                px_int = int(px)
                py_int = int(py)
                u = px - px_int
                v = py - py_int

                A = np.matrix([[base_function(u+1,a)],[base_function(u,a)],[base_function(u-1,a)],[base_function(u-2,a)]])
                C = np.matrix([base_function(v+1,a),base_function(v,a),base_function(v-1,a),base_function(v-2,a)])
                B = np.matrix([
                    [img[py_int-1,px_int-1][c],img[py_int-1,px_int][c],img[py_int-1,px_int+1][c],img[py_int-1,px_int+2][c]],
                    [img[py_int,px_int-1][c],img[py_int,px_int][c],img[py_int,px_int+1][c],img[py_int,px_int+2][c]],
                    [img[py_int+1,px_int-1][c],img[py_int+1,px_int][c],img[py_int+1,px_int+1][c],img[py_int+1,px_int+2][c]],
                    [img[py_int+2,px_int-1][c],img[py_int+2,px_int][c],img[py_int+2,px_int+1][c],img[py_int+2,px_int+2][c]]
                ])
                new_img[j, i][c] = np.dot(np.dot(C, B), A)
    return new_img


In [45]:
sacle = 8
path = "C:/Users/1/Desktop/picture/Venti/Venti.jpg"
img = cv2.imread(path)
new_img_BICBIC = bicubic(img, sacle)
cv2.imwrite("C:/Users/1/Desktop/picture/Venti/Venti_H_INTER_BICBIC.jpg", new_img_BICBIC)
print('INTER_BICUBIC-双三次插值 PSNR',format(psnr(img_h, new_img_BICBIC)))

INTER_BICUBIC-双三次插值 PSNR 20.30545923432966
Doing bicubic
图像的尺寸为 (300, 300, 3)
INTER_BICUBIC-双三次插值 PSNR 20.30545923432966
